In [52]:
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard
import cv2
from keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img,array_to_img
import h5py

In [78]:
# f = h5py.File('BRATS2015.h5', 'r')
# # (x_train, _), (x_test, _) = mnist.load_data()

# # x_train = x_train.astype('float32') / 255.
# # x_test = x_test.astype('float32') / 255.
# # x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
# # x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format
# x_train = f['x_train']
# noise_factor = 0.5
# print(x_train.shape)
# x = cv2.cvtColor(x_train[1], cv2.COLOR_BGR2GRAY)

# print(x.shape)
# x_train_noisy = x = x.reshape(x.shape + (1,))  # converting to a Numpy array
img = load_img('trainingSet/frame2.jpg')

x = img_to_array(img)  # creating a Numpy array with shape
x = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
# x= x.reshape(28,28,-1)
# x = x.reshape((1,) + x.shape)  # converting to a Numpy array
print(x.shape)
x_train =x
x_train_noisy = x + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x.shape) 
# x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

# x_train_noisy = np.clip(x_train_noisy, 0., 1.)
# x_test_noisy = np.clip(x_test_noisy, 0., 1.)

(211, 246)


In [ ]:
# n = 10

# plt.figure(figsize=(20, 2))
# for i in range(n):
#     ax = plt.subplot(1, n, i+1)
#     plt.imshow(x_test_noisy[i].reshape(28, 28))
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
# plt.show()
# print('hello worlds')

In [79]:
input_img = Input(x.shape)  # adapt this if using `channels_first` image data format

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (7, 7, 32)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(x_train_noisy, x_train,
                epochs=1,
                batch_size=128,
                shuffle=True,
#                 validation_data=(x_test_noisy, x_test),
#                 callbacks=[TensorBoard(log_dir='/tmp/tb', histogram_freq=0, write_graph=False)]
               )
# decoded_imgs = autoencoder.predict(x_test_noisy)

ValueError: Input 0 is incompatible with layer conv2d_74: expected ndim=4, found ndim=3

In [ ]:
plt.figure(figsize=(20, 4))
for i in range(10):
    # original
    plt.subplot(2, 10, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    plt.axis('off')
 
    # reconstruction
    plt.subplot(2, 10, i + 1 + 10)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    plt.axis('off')
 
plt.tight_layout()
plt.show()